In [56]:
import os
from params import PARAMS
import pickle
import numpy as np
import json

In [57]:
import pandas as pd

def eval_runs(model_names, dataset_names, random_seeds):
    final_results = []
    for model_name in model_names:
        for dataset_name in dataset_names:
            subject_idc = json.load(open(os.path.join('./dataset/train', dataset_name + '.json'), 'r'))
            accs = []
            precs_macro = []
            precs_weighted = []
            recalls_macro = []
            recalls_weighted = []
            aurocs = []
            n_trainable_params = []

            for random_seed in random_seeds:
                path = os.path.join('output', model_name, dataset_name.rstrip('.pickle').lstrip('./dataset/train/'), str(random_seed), 'results.pickle')
                results = pickle.load(open(path, 'rb'))

                accs.append(results['cr']['accuracy'])
                precs_macro.append(results['cr']['macro avg']['precision'])
                precs_weighted.append(results['cr']['weighted avg']['precision'])
                recalls_macro.append(results['cr']['macro avg']['recall'])
                recalls_weighted.append(results['cr']['weighted avg']['recall'])
                aurocs.append(results['auroc'])
                n_trainable_params.append(results['n_params'])
            result_df = pd.DataFrame({'model_name': [model_name for i in random_seeds], 'dataset_name': [dataset_name for i in random_seeds],'train_idc': [subject_idc['train_idc'] for i in random_seeds], 'test_idc': [subject_idc['test_idc'] for i in random_seeds], 'random_seed': random_seeds, 'accuracy': accs, 'precision_macro': precs_macro, 'precision_weighted': precs_weighted, 'recall_macro': recalls_macro, 'recall_weighted': recalls_weighted, 'AUROC': aurocs, 'n_params': n_trainable_params})
            final_results.append(result_df)
    final_results = pd.concat(final_results).reset_index(drop=True)
    final_results.to_csv('./output/results.csv', index=False)

    std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
    std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                        'precision_macro': 'precision_macro_std', 
                        'precision_weighted': 'precision_weighted_std', 
                        'recall_macro': 'recall_macro_std', 
                        'recall_weighted': 'recall_weighted_std'})
    std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
    mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
    results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
    results_per_dataset.to_csv('./output/results_per_dataset.csv')

    std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
    std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                        'precision_macro': 'precision_macro_std', 
                        'precision_weighted': 'precision_weighted_std', 
                        'recall_macro': 'recall_macro_std', 
                        'recall_weighted': 'recall_weighted_std'})
    std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
    mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
    results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
    results_per_model.to_csv('./output/results_per_model.csv')
    print(results_per_model)

In [58]:
model_names = ['imagine_fcn', 'imagine_cnn', 'imagine_rnn', 'imagine_gcn', 'imagine_gcn_auto']
# model_names = ['imagine_gcn_auto']
random_seeds = PARAMS['RANDOM_SEEDS']
print(random_seeds)
dataset_names = [f'cross_subject_data_{i}_5_subjects' for i in range(5)]
final_results = eval_runs(model_names, dataset_names, random_seeds)
# for k, v in final_results.items():
#     print(k)
#     print(v['accuracy'])

[688933, 890203, 517391, 195645, 705223]
                  accuracy  precision_macro  precision_weighted  recall_macro  \
model_name                                                                      
imagine_cnn       0.262778         0.263174            0.263986      0.262811   
imagine_fcn       0.250000         0.164931            0.164624      0.250250   
imagine_gcn       0.250486         0.109793            0.110008      0.250244   
imagine_gcn_auto  0.254514         0.256606            0.257395      0.254605   
imagine_rnn       0.257014         0.257399            0.257379      0.257678   

                  recall_weighted     AUROC  n_params  accuracy_std  \
model_name                                                            
imagine_cnn              0.262778  0.508606  371220.0      0.022851   
imagine_fcn              0.250000  0.500149  354308.0      0.011094   
imagine_gcn              0.250486  0.500152  354308.0      0.011201   
imagine_gcn_auto         0.254514  0

In [59]:
final_results = pd.read_csv('./output/results.csv')

std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
results_per_dataset.reset_index()

,model_name,dataset_name,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
0,imagine_cnn,cross_subject_data_0_5_subjects,0.273611,0.274331,0.274696,0.274681,0.273611,0.516446,371220.0,0.009397,0.010679,0.010861,0.009379,0.009397
1,imagine_cnn,cross_subject_data_1_5_subjects,0.271875,0.275459,0.277549,0.271058,0.271875,0.514350,371220.0,0.023981,0.028998,0.029688,0.024141,0.023981
2,imagine_cnn,cross_subject_data_2_5_subjects,0.255556,0.255148,0.255847,0.255302,0.255556,0.503593,371220.0,0.012362,0.011624,0.011758,0.012121,0.012362
3,imagine_cnn,cross_subject_data_3_5_subjects,0.231944,0.232301,0.233127,0.231945,0.231944,0.487996,371220.0,0.009790,0.010676,0.010906,0.009909,0.009790
4,imagine_cnn,cross_subject_data_4_5_subjects,0.280903,0.278630,0.278712,0.281072,0.280903,0.520645,371220.0,0.017508,0.018236,0.017921,0.018128,0.017508
5,imagine_fcn,cross_subject_data_0_5_subjects,0.259028,0.274794,0.275741,0.259282,0.259028,0.506249,354308.0,0.017248,0.136167,0.138399,0.012419,0.017248
6,imagine_fcn,cross_subject_data_1_5_subjects,0.244792,0.164284,0.164423,0.244009,0.244792,0.495988,354308.0,0.003248,0.099520,0.100144,0.009770,0.003248
7,imagine_fcn,cross_subject_data_2_5_subjects,0.252083,0.063021,0.063546,0.250000,0.252083,0.500000,354308.0,0.000776,0.000194,0.000392,0.000000,0.000776
8,imagine_fcn,cross_subject_data_3_5_subjects,0.240625,0.186821,0.184777,0.242572,0.240625,0.494951,354308.0,0.010168,0.075875,0.077864,0.007137,0.010168
9,imagine_fcn,cross_subject_data_4_5_subjects,0.253472,0.135736,0.134633,0.255385,0.253472,0.503556,354308.0,0.007764,0.108280,0.107803,0.011992,0.007764


In [60]:
std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
results_per_model

,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
model_name,,,,,,,,,,,,
imagine_cnn,0.262778,0.263174,0.263986,0.262811,0.262778,0.508606,371220.0,0.022851,0.023984,0.024136,0.023036,0.022851
imagine_fcn,0.250000,0.164931,0.164624,0.250250,0.250000,0.500149,354308.0,0.011094,0.112295,0.113066,0.010815,0.011094
imagine_gcn,0.250486,0.109793,0.110008,0.250244,0.250486,0.500152,354308.0,0.011201,0.079148,0.079149,0.008116,0.011201
imagine_gcn_auto,0.254514,0.256606,0.257395,0.254605,0.254514,0.503131,358404.0,0.013448,0.016253,0.016562,0.013746,0.013448
imagine_rnn,0.257014,0.257399,0.257379,0.257678,0.257014,0.505027,357556.0,0.023106,0.024064,0.023958,0.022840,0.023106


In [61]:
path = 'output/imagine_cnn/cross_subject_data_0/195645/results.pickle'
results = pickle.load(open(path, 'rb'))

In [62]:
results['history']['val_acc']

[0.28912869095802307,
 0.2906447947025299,
 0.30393293499946594,
 0.29135823249816895,
 0.28877195715904236,
 0.29626327753067017,
 0.2916257977485657,
 0.294568806886673,
 0.2949255406856537,
 0.30009809136390686,
 0.2949255406856537,
 0.29474717378616333,
 0.2933202385902405,
 0.30214929580688477,
 0.29082316160202026,
 0.28029966354370117,
 0.29296353459358215,
 0.2868099510669708,
 0.29465797543525696,
 0.29421207308769226,
 0.2916257977485657,
 0.2868991196155548,
 0.2931418716907501,
 0.29055559635162354,
 0.2914474308490753,
 0.28377774357795715,
 0.2889503240585327,
 0.29055559635162354,
 0.28841522336006165,
 0.29251760244369507,
 0.29180416464805603,
 0.2860964834690094,
 0.28663158416748047,
 0.2899313271045685,
 0.2874342203140259,
 0.2857397496700287,
 0.29171496629714966,
 0.28288593888282776,
 0.29091233015060425,
 0.28636404871940613,
 0.2914474308490753,
 0.2894854247570038,
 0.2901988625526428,
 0.2857397496700287,
 0.2838669419288635,
 0.28618568181991577,
 0.2915365